# LaTeX table of dynamic rupture model result

Generate the table of dynamic rupture model results.

2025.4.1 Kurama Okubo


In [15]:
import os
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import warnings
import time

import scipy.integrate as integrate
import seaborn as sns 

%load_ext autoreload
%autoreload 2

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 11
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True

plt.rcParams['axes.linewidth'] = 0.75

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Result of dynamic rupture simulation

In [16]:
# dynamic rupture parameters
a_patch = 4.0e-3 # patch radius without margine
rupturetype = "pulse" # "crack": without self-healing or "pulse": with self-healing
p_dcscaleexp = 0.6 #0.55 #0.54 #0.56 #0.555  # #0.475 #0.55 #0.575 #0.65 #0.8 #0.65 #0.7
bgbeta= 0.35 #0.4
sig_n = 6e6
    
casestr = f"a={a_patch*1e3:.2f}_ruptype={rupturetype}_pdcscaling={p_dcscaleexp:.3f}_sn={sig_n/1e6:.1f}MPa_hatfr=0.3_bgbeta={bgbeta:.2f}"
print("read dynamic rupture result:"+casestr)
# read STF best-fit associated with the dynamic rupture model
df_dynrup_sourceparam = pd.read_csv(f"../data/dynrup_bestfit_sourceparam_{casestr}_master.csv", index_col=None)
df_dynrup_sourceparam = df_dynrup_sourceparam.set_index("event_id")
df_dynrup_sourceparam.index = [f"M{x}" for x in df_dynrup_sourceparam.index]

read dynamic rupture result:a=4.00_ruptype=pulse_pdcscaling=0.600_sn=6.0MPa_hatfr=0.3_bgbeta=0.35


In [17]:
df_dynrup_sourceparam

,M0_rec,M0_bestfit,Tw_bestfit,Tshift_bestfit
M24,0.064238,0.066640,0.000002,1.422658e-07
M50,0.300888,0.309173,0.000003,3.634125e-07
M52,0.454187,0.465928,0.000003,3.405351e-07
M72,0.806391,0.821574,0.000003,2.727666e-07
M129,1.146419,1.149405,0.000003,2.176500e-07


In [18]:
# Extract M0 and Tw estimated by fitting cosine STF
df_dynrup_sourceparam_selected = df_dynrup_sourceparam[["M0_bestfit", "Tw_bestfit",]].copy()

In [19]:
df_dynrup_sourceparam_selected.rename(
    columns={"M0_bestfit":"$M_0^{\\text{model}}$", 
             "Tw_bestfit":"$T_w^{\\text{model}}$",
            },
    inplace=True)

In [20]:
df_dynrup_sourceparam_selected

,$M_0^{\text{model}}$,$T_w^{\text{model}}$
M24,0.066640,0.000002
M50,0.309173,0.000003
M52,0.465928,0.000003
M72,0.821574,0.000003
M129,1.149405,0.000003


## unit Convert 

In [21]:
s2mus = lambda x: x*1e6

In [22]:
# Unit conversion
formatters_targets = {"$M_0^{\\text{model}}$": lambda x: '%.2f' % x,
                      "$T_w^{\\text{model}}$": lambda x: '%.2f' % s2mus(x),
                     }

In [23]:
latextable_sourceparam = df_dynrup_sourceparam_selected.to_latex(header=True, column_format=f"l"+"c"*(len(df_dynrup_sourceparam_selected.columns)), formatters= formatters_targets)


In [24]:
print(latextable_sourceparam)

\begin{tabular}{lcc}
\toprule
 & $M_0^{\text{model}}$ & $T_w^{\text{model}}$ \\
\midrule
M24 & 0.07 & 2.47 \\
M50 & 0.31 & 2.57 \\
M52 & 0.47 & 2.59 \\
M72 & 0.82 & 2.61 \\
M129 & 1.15 & 2.63 \\
\bottomrule
\end{tabular}



# Evaluate the difference between $M_0^{rec}$ and $M_0^{model}$

We check if the $M_0$ obtained by the numerical integration of $\dot{M}_0^{rec}$ is close to the cosine fit STF, $M_0^{model}$.

In [25]:
df_dynrup_sourceparam.loc[:, "M0compare",] = df_dynrup_sourceparam.apply(lambda x: x.M0_rec/x.M0_bestfit, axis=1 )

In [26]:
df_dynrup_sourceparam

,M0_rec,M0_bestfit,Tw_bestfit,Tshift_bestfit,M0compare
M24,0.064238,0.066640,0.000002,1.422658e-07,0.963962
M50,0.300888,0.309173,0.000003,3.634125e-07,0.973204
M52,0.454187,0.465928,0.000003,3.405351e-07,0.974800
M72,0.806391,0.821574,0.000003,2.727666e-07,0.981520
M129,1.146419,1.149405,0.000003,2.176500e-07,0.997402


In [27]:
df_dynrup_sourceparam.M0compare * 1e2

M24     96.396214
M50     97.320409
M52     97.479954
M72     98.152041
M129    99.740161
Name: M0compare, dtype: float64

The comparison indicates that the difference does not exceed 4%. Thus, the cosine fitting estimates are similar to the numerical integration of $\dot{M}_0^{rec}$.